In [2]:
from dolfin import *

\begin{align}
-\Delta u = \lambda u, \quad \Omega\\
u = 0, \quad \partial \Omega
\end{align}

In [4]:
mesh = UnitSquareMesh(32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS or x[1] > 1.0 - DOLFIN_EPS

v0 = Constant(0.0)
bc = DirichletBC(V, v0, boundary)

In [5]:
u = TrialFunction(V)
v = TestFunction(V)

In [6]:
dummy = inner(Constant(1), v)*dx
a = inner(grad(u), grad(v))*dx
asm = SystemAssembler(a, dummy, bc)
A = PETScMatrix(); asm.assemble(A)

In [7]:
b = inner(u, v)*dx
asm = SystemAssembler(b, dummy)
B = PETScMatrix(); asm.assemble(B)
bc.zero(B)

In [8]:
num_eigenfunctions = 15

In [9]:
%%time
solver = SLEPcEigenSolver(A, B)
solver.parameters["solver"] = "krylov-schur"
solver.parameters["spectrum"] = "target magnitude"
solver.parameters["problem_type"] = "gen_hermitian"
solver.parameters["spectral_transform"] = "shift-and-invert"
solver.parameters["spectral_shift"] = 10.
solver.solve(num_eigenfunctions)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 57.5 ms


In [16]:
solver.get_eigenpair(0)

(19.786792290191517,
 0.0,
 <dolfin.cpp.la.PETScVector at 0x7fdcdf6d8fe0>)

In [21]:
eigenmodes = File("eigenmodes.pvd")
eigenfunction = Function(V, name="Eigenfunction")
print(solver.get_number_converged())
for i in range(solver.get_number_converged()):
    (r, _, rv, _) = solver.get_eigenpair(i)
    eigenfunction.vector().zero()
    eigenfunction.vector().axpy(1, rv)
    eigenmodes << eigenfunction
    File(f'Eigenfunction_{i}.xml')  << eigenfunction

15
